⚠️ On HSLU GPUHUB You must use the server template __CVAI admin__ ⚠️

# Setup (run each time)

## Install packages

Check GPU + Cuda version

In [ ]:
!nvidia-smi

Upgrade httpx

In [ ]:
!pip install -U httpx==0.24.1

Install xformers to speed up inference

In [ ]:
!pip install -U xformers==0.0.25 huggingface-hub==0.22.0

Upgrade torch, etc...

In [ ]:
!pip install -U torch==2.2.1+cu121 torchvision==0.17.1+cu121 --index-url https://download.pytorch.org/whl/cu121

In [ ]:
!pip install torchsde

Check if GPU is working

In [ ]:
import torch
torch.cuda.is_available()

## Declare var + functions

In [ ]:
import os
import git
import shutil
import huggingface_hub
from pathlib import Path

# Home path (only works on Linux)
# https://stackoverflow.com/a/58988310 (root path)
# home_dir = Path(os.readlink('/proc/%s/cwd' % os.environ['JPY_PARENT_PID']))
home_dir = Path(os.getcwd())

# Paths
sd_dir = home_dir / 'stable-diffusion-webui'
sd_model_dir = sd_dir / 'models'
sd_extensions_dir = sd_dir / 'extensions'
fooocus_dir = home_dir / 'Fooocus'
fooocus_api_dir = home_dir / 'Fooocus-API'
comfyui_dir = home_dir / 'ComfyUI'

In [ ]:
# Clone a repo
def clone_repo(git_url, target_dir):
    
    # Extract repo name and create path
    repo_name = git_url.split('/')[-1]

    # Create path
    repo_dir = target_dir / repo_name

    # Clone if does not exist
    if not repo_dir.exists():
        git.Repo.clone_from(git_url, repo_dir)

# SD-WEBUI

## Install repo (run once)

**⚠️** Warning: only run the next cell if you want to delete the existing copy of sd-webui **⚠️**

In [ ]:
answer = input("-> Are you sure you want to delete 'stable-diffusion-webui' [y | n]: ")
if answer == 'y':
    if sd_dir.exists():
        shutil.rmtree(sd_dir)
        print('Folder deleted')
    else:
        print('Folder not found')
else:
    print('Skipping')

Clone and cd into repo

In [ ]:
clone_repo('https://github.com/AUTOMATIC1111/stable-diffusion-webui', home_dir)

#### 1. Manually modify the file `stable-diffusion/webui.sh`

Change `use_venv=1` to `use_venv=0`

#### 2. Manually modify the file `stable-diffusion/webui-user.sh`

change `#export COMMANDLINE_ARGS=""` to  `export COMMANDLINE_ARGS="--xformers --share --api --update-check --update-all-extensions --enable-insecure-extension-access --opt-sdp-attention"`

Dont forget to uncomment the line!! (delete the `#`)  

NB: We also need to set `disable_mmap_load_safetensors` in the settings on the GUI

## Install extensions (run once)

You can find extensions via the UI then copy paste their repo here

In [ ]:
# Use urls without the .git
git_urls = {
    'https://github.com/zanllp/sd-webui-infinite-image-browsing',
    'https://github.com/Mikubill/sd-webui-controlnet',
    'https://github.com/butaixianran/Stable-Diffusion-Webui-Civitai-Helper'
}

# Iterate through git repos and clone them
for url in git_urls:
    clone_repo(url, sd_extensions_dir)

## 📍 Run

You might need to download some models before starting the UI the first time, see section **"Download additional models"**

In [ ]:
!cd {sd_dir} && ./webui.sh

# FOOOCUS

## Install repo (run once)

**⚠️** Warning: only run the next cell if you want to delete the existing copy of Fooocus **⚠️**

In [ ]:
answer = input("-> Are you sure you want to delete 'Fooocus' [y | n]: ")
if answer == 'y':
    if fooocus_dir.exists():
        shutil.rmtree(fooocus_dir)
        print('Folder deleted')
    else:
        print('Folder not found')
else:
    print('Skipping')

Clone repo

In [ ]:
clone_repo('https://github.com/lllyasviel/Fooocus', home_dir)

Start Fooocus once to initialize all files, generate a few image and stop the execution of the cell.

In [ ]:
!python {fooocus_dir}/entry_with_update.py --share

In [ ]:
str(home_dir)

#### Manually modify the file `Fooocus/config.txt`

Assuming the `home_dir` displayed above ⬆️ is `/home/jovyan` (it might be different than `/home/jovyan`) change:

```
    "path_checkpoints": [
        "/home/jovyan/Fooocus/models/checkpoints"
    ],
    "path_loras": [
        "/home/jovyan/Fooocus/models/loras"
    ],
```
to 
```
    "path_checkpoints": [
        "/home/jovyan/stable-diffusion-webui/models/Stable-diffusion"
    ],
    "path_loras": [
        "/home/jovyan/stable-diffusion-webui/models/Lora"
    ],
```

Mode details here: https://github.com/lllyasviel/Fooocus?tab=readme-ov-file#customization

## 📍 Run

In [ ]:
!python {fooocus_dir}/entry_with_update.py --share --always-download-new-model --disable-offload-from-vram --always-high-vram --attention-pytorch

# Download additional models (run once)

Both sd-webui and Fooocus should download a default model the first time you start them but you will need to download some additional models for ControlNet

In [ ]:
# ControlNet (There's a LOT here, hopefully we have enough disk space 🤞)
local_dir = sd_model_dir / 'ControlNet'

# ControlNet 1.1 
huggingface_hub.snapshot_download(
    repo_id='lllyasviel/ControlNet-v1-1',
    allow_patterns='*.pth',
    local_dir=local_dir
)
# ControlNet XL
huggingface_hub.snapshot_download(
    repo_id='lllyasviel/sd_control_collection',
    local_dir=local_dir
)

We also download the default sd1.5 and xl models in case they are not there

In [ ]:
# SD models
local_dir = sd_model_dir / 'Stable-diffusion'

# SD 1.5
repo_id='runwayml/stable-diffusion-v1-5'
filename = 'v1-5-pruned-emaonly.safetensors'
huggingface_hub.hf_hub_download(
    repo_id=repo_id,
    filename=filename,
    local_dir=local_dir
)

# SDXL
repo_id='stabilityai/stable-diffusion-xl-base-1.0'
# Base model
filename = 'sd_xl_base_1.0.safetensors'
huggingface_hub.hf_hub_download(
    repo_id=repo_id,
    filename=filename,
    local_dir=local_dir
)
# VAE
filename = 'sd_xl_base_1.0_0.9vae.safetensors'
huggingface_hub.hf_hub_download(
    repo_id=repo_id,
    filename=filename,
    local_dir=local_dir
)

A nice / fun SD model (1.5)

In [ ]:
# SD models
local_dir = sd_model_dir / 'Stable-diffusion'

# https://civitai.com/models/7371/rev-animated
repo_id='s6yx/ReV_Animated'
filename = 'rev_1.2.2.safetensors'
subfolder = 'rev_1.2.2'

huggingface_hub.hf_hub_download(
    repo_id=repo_id,
    filename=filename,
    subfolder=subfolder,
    local_dir=local_dir
)

We move it to the root of the sd models dir. We don't really have to but it's "cleaner" this way

In [ ]:
!mv {local_dir}/{subfolder}/{filename} {local_dir}

"Manual" download of turbovisionxl and Realities edge (2 versions)

In [ ]:
!curl --progress-bar https://drive.switch.ch/index.php/s/dL2tJjvuO9rgH95/download -o {sd_model_dir}/Stable-diffusion/turbovisionxlSuperFastXLBasedOnNew_alphaV0101Bakedvae.safetensors

In [ ]:
!curl --progress-bar https://drive.switch.ch/index.php/s/6yBKEF33YIL8m4q/download -o {sd_model_dir}/Stable-diffusion/Realities_Edge_XL_Lightning_Turbo_V7.safetensors

A couple of autoencoders used for SD

In [ ]:
# SD models
local_dir = sd_model_dir / 'VAE'

# SD VAE
repo_id='stabilityai/sd-vae-ft-mse-original'
filename = 'vae-ft-mse-840000-ema-pruned.safetensors'
huggingface_hub.hf_hub_download(
    repo_id=repo_id,
    filename=filename,
    local_dir=local_dir
)

# SD XL VAE
repo_id='madebyollin/sdxl-vae-fp16-fix'
filename = 'sdxl.vae.safetensors'
huggingface_hub.hf_hub_download(
    repo_id=repo_id,
    filename=filename,
    local_dir=local_dir
)

# Comfyui

## Install repo (run once)

Clone Repo

In [ ]:
clone_repo('https://github.com/comfyanonymous/ComfyUI', home_dir)

In [ ]:
str(sd_dir)

Manually modify the file `ComfyUI/extra_model_paths.yaml.example`

1) Change `base_path: path/to/stable-diffusion-webui/` with the path from our install (see the value of the var `sd_dir` ⬆️)
2) Rename file to `ComfyUI/extra_model_paths.yaml`

In [ ]:
# Manager for custom nodes and then some...
clone_repo('https://github.com/ltdrdata/ComfyUI-Manager', comfyui_dir / 'custom_nodes')
# ComfyUI-Custom-Scripts
clone_repo('https://github.com/pythongosssss/ComfyUI-Custom-Scripts', comfyui_dir / 'custom_nodes')
# Animate diff
clone_repo('https://github.com/Kosinkadink/ComfyUI-AnimateDiff-Evolved', comfyui_dir / 'custom_nodes')

### Models for Animate diff

In [ ]:
# Download models for AnimateDiff https://github.com/Kosinkadink/ComfyUI-AnimateDiff-Evolved#how-to-use
repo_id='guoyww/animatediff'

# Motion modules
local_dir = comfyui_dir / 'custom_nodes/ComfyUI-AnimateDiff-Evolved/models'

huggingface_hub.snapshot_download(
    repo_id=repo_id,
    local_dir=local_dir,
    allow_patterns="mm_sd*.ckpt"
)

# Motion Lora
local_dir = comfyui_dir / 'custom_nodes/ComfyUI-AnimateDiff-Evolved/motion_lora'

huggingface_hub.snapshot_download(
    repo_id=repo_id,
    local_dir=local_dir,
    allow_patterns="v2_lora*.ckpt"
)

We will need to install some nodes manually afterwards, see:  
https://github.com/Kosinkadink/ComfyUI-AnimateDiff-Evolved#animatediff-for-comfyui

## 📍 Run

In [ ]:
os.chdir(comfyui_dir)

In [ ]:
!python main.py